In [49]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import gspread
from google.colab import auth

#First Authenticate google colab
auth.authenticate_user()

#Authenticate google sheet
from google.auth import default
creds, _ = default()

gs = gspread.authorize(creds)

#Get google sheet
sh = gs.open("Dados compilados sobre aves catalogadas no Brasil")
sh_regions = sh.worksheet("Regiões com espécies em comum com o Brasil")
sh_birds = sh.worksheet("Espécies de aves no Brasil")
sh_birds_states = sh.worksheet("Espécies de aves por estado do Brasil")

# **Regiões com espécies em comum com o Brasil**

In [ ]:
#Extract html
page_regions = requests.get('https://avibase.bsc-eoc.org/compare_regions.jsp?region=BR&list=clements')
bs = BeautifulSoup(page_regions.text, 'html.parser')

In [25]:
#Transform table into list
tb_regions = bs.find(attrs={'id':'compareRegions'}).find_all('tbody')[0].find_all('tr')
tb_regions = [ [td.string for td in tr.find_all('td')] for tr in tb_regions]

tb_regions_headers = bs.find(attrs={'id':'compareRegions'}).find('thead').find('tr').find_all('th')
tb_regions_headers = [ x.string for x in tb_regions_headers ]

tb_regions = [tb_regions_headers] + tb_regions

In [37]:
#Filter columns with pandas
df = pd.DataFrame(tb_regions[1:], columns=tb_regions[0])

df = df[['Region2','Shared','% shared/region1','%shared/region2']]

data = df.fillna(-1).values.tolist()

In [52]:
#Update Google sheet
sh_regions.append_rows(data,table_range='A2')

{'spreadsheetId': '1jkUNttrcJa_U75cVSiNhHXGSfWZbFmdK8E5CcJb7XMM',
 'updates': {'spreadsheetId': '1jkUNttrcJa_U75cVSiNhHXGSfWZbFmdK8E5CcJb7XMM',
  'updatedRange': "'Regiões com espécies em comum com o Brasil'!A2:D243",
  'updatedRows': 242,
  'updatedColumns': 4,
  'updatedCells': 968}}